<a href="https://colab.research.google.com/github/marinebon/HackingLimno2025/blob/main/02_Sample_environmental_data_at_each_occurrence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [38]:
# ==============================================================================
# === directory setup
# ==============================================================================
# === If using google colab and google drive:
# TODO: is there a way to do this without granting excessive permissions?
from google.colab import drive
drive.mount('/content/drive')

# NOTE: Before running you must create a folder in your google drive
#       matching the directory name here `GSoC_SDM_Project`.
PROJECT_DIR = '/content/drive/MyDrive/GSoC_SDM_Project'

# === if using local machine
# PROJECT_DIR = './'
#import os
#if not os.path.exists(PROJECT_DIR):
#    os.makedirs(PROJECT_DIR)
# ==============================================================================
# ==============================================================================
# === spatial coverage
# ==============================================================================
# # === FL
LATMIN = 24.0
LATMAX = 30.7
LONMIN = -87.9
LONMAX = -79.5
# === FL Keys
# LATMIN = 24.11637699635014
# LATMAX = 26.11949526731449
# LONMIN = -82.51572158798965
# LONMAX = -79.61106009492724
# ==============================================================================

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
!pip install rasterio pandas geopandas matplotlib  requests xarray netCDF4

mount google drive for saving result

In [40]:
# read parquet file of occurrences
# see step 01 https://github.com/marinebon/HackingLimno2025

import pandas as pd
%matplotlib inline
df_clean = pd.read_parquet(f'{PROJECT_DIR}/occurrences.parquet')

Function to prepare data for environmental sampling

In [43]:
import rasterio
import xarray as xr
import numpy as np
from typing import Union, List, Tuple, Optional, Dict

def add_environmental_data(
    df: pd.DataFrame,
    raster_path: str,
    column_name: str = 'temperature',
    lat_col: str = 'decimalLatitude',
    lon_col: str = 'decimalLongitude'
) -> pd.DataFrame:

    df_result = df.copy()

    # TODO: change to use xarray
    with rasterio.open(raster_path) as src:
        coords = [(row[lon_col], row[lat_col]) for _, row in df.iterrows()]
        sampled_values = list(src.sample(coords))
        band_index = 0  # 0 is surface
        values = [val[band_index] if val[band_index] != src.nodata else np.nan for val in sampled_values]
        df_result[column_name] = values

    print(f"Added {column_name}: {df_result[column_name].notna().sum()}/{len(df_result)} valid values")
    return df_result

# multidecadal environmental data from NOAA World Ocean Atlas
# https://www.ncei.noaa.gov/access/world-ocean-atlas-2023/

# TODO: what about depth?df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_all_i00_01.nc_i_mn.tif', 'i')
df_with_env = df_clean
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_all_i00_01.nc_i_mn.tif', 'i')
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_all_i00_01.nc_i_sd.tif', 'i_sd')
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_all_n00_01.nc_n_mn.tif', 'n')
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_all_n00_01.nc_n_sd.tif', 'n_sd')
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_all_o00_01_o_sd.tif', 'o')
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_all_o00_01_o_mn.tif', 'o_sd')
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_all_O00_01.nc_A_mn.tif', 'O')
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_all_O00_01.nc_A_sd.tif', 'O_sd')
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_all_p00_01.nc_p_mn.tif', 'p')
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_all_p00_01.nc_p_sd.tif', 'p_sd')
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_decav_t00_04_t_mn.tif', 't')
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_decav_t00_04_t_sd.tif', 't_sd')


print(df_with_env.head())

df_with_env.to_parquet(f'{PROJECT_DIR}/occurrences_and_environment.parquet', engine='pyarrow', index=False)

Added i: 19078/30535 valid values
Added i_sd: 9665/30535 valid values
Added n: 9665/30535 valid values
Added n_sd: 9665/30535 valid values
Added o: 22881/30535 valid values
Added o_sd: 22881/30535 valid values
Added O: 22881/30535 valid values
Added O_sd: 22881/30535 valid values
Added p: 22816/30535 valid values
Added p_sd: 22816/30535 valid values
Added t: 28498/30535 valid values
Added t_sd: 28498/30535 valid values
   decimalLatitude  decimalLongitude occurrenceStatus  date_year  \
0         25.04470         -80.39480          present     1999.0   
1         24.95167         -80.45990          present     2014.0   
2         25.76970         -80.08830          present     2016.0   
3         24.60800         -82.94400          present     2016.0   
4         25.81166         -80.10945          present     2022.0   

       date_mid         i      i_sd         n      n_sd         o        o_sd  \
0  9.308736e+11       NaN       NaN       NaN       NaN       NaN         NaN   
1  1.4

In [44]:
# sample data using xarray and thredds
# NOTE: this takes a long time to run.
import xarray as xr

# url = 'https://www.ncei.noaa.gov/thredds-ocean/fileServer/woa23/DATA/temperature/netcdf/decav/0.25/woa23_decav_t00_04.nc'
data_urls = {
    't_an': 'https://www.ncei.noaa.gov/thredds-ocean/dodsC/woa23/DATA/temperature/netcdf/decav/0.25/woa23_decav_t00_04.nc'
}
for varName, data_url in data_urls.items():
    ds = xr.open_dataset(data_url, decode_times=False)
    var_data = ds[varName]

    def extract_value(row):
        selected = var_data.sel(
            lat=row['decimalLatitude'],
            lon=row['decimalLongitude'],
            method='nearest'
        ).isel(depth=0)  # Select the first depth level (surface)

        # Remove extra dimensions
        selected = selected.squeeze()

        # Debugging
        if selected.size != 1:
            print(f"Unexpected shape for {varName} at lat={row['decimalLatitude']}, lon={row['decimalLongitude']}: {selected.shape}")

        return selected.item()  # Will work if single value

    print(varName)
    df_clean[varName] = df_clean.apply(extract_value, axis=1)

print(df_clean.head())


t_an


KeyboardInterrupt: 